<a href="https://colab.research.google.com/github/chw8207/NLP-study/blob/main/%EC%98%90%ED%94%84_%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from argparse import Namespace
from collections import Counter
import json
import os
import re
import string

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import tqdm

### 데이터셋

In [ ]:
class ReviewDataset(Dataset) : 
  def __init__(self, review_df, vetorizer) : 
    """
        매개변수:
            review_df (pandas.DataFrame): 데이터셋
            vectorizer (ReviewVectorizer): ReviewVectorizer 객체
    """
    self.review_df = review_df
    self.vectorizer = vectorizer

    self.train_df = self.review_df[self.review_df.split=='train']
    self.train_size = len(self.train_df)

    self.val_df = self.review_df[self.review_df.split=='val']
    self.val_size = len(self.val_df)

    self._lookup_dict = {'train':(self.train_df, self.train_size),
                         'val':(self.val_df, self.val_size),
                         'test':(self.test_df, self.test_size)}
    self.set_split('train')

  @classmethod  # 시작 메소드
  def load_dataset_and_make_vectorizer(cls, review_csv) : 
    """ 데이터셋을 로드하고 새로운 ReviewVectorizer 객체를 만듭니다
        
        매개변수:
            review_csv (str): 데이터셋의 위치
        반환값:
            ReviewDataset의 인스턴스
    """
    review_df = pd.read_csv(review_csv)
    return cls(review_df, ReviewVectorizer.from_dataframe(review_df))

  @classmethod 
  def load_dataset_and_load_vectorizer(cls, review_csv, vectorizer_filepath) : 
    """ 데이터셋을 로드하고 새로운 ReviewVectorizer 객체를 만듭니다.
        캐시된 ReviewVectorizer 객체를 재사용할 때 사용합니다.
        
        매개변수:
            review_csv (str): 데이터셋의 위치
            vectorizer_filepath (str): ReviewVectorizer 객체의 저장 위치
        반환값:
            ReviewDataset의 인스턴스
    """
    review_df = pd.read_csv(review_csv)
    vectorizer = cls.load_vectorizer_only(vectorizer_filepath)
    return cls(review_df, vectorizer)

  @staticmethod
  def load_vectorizer_only(vectorizer_filepath) : 
    """ 파일에서 ReviewVectorizer 객체를 로드하는 정적 메서드
        
        매개변수:
            vectorizer_filepath (str): 직렬화된 ReviewVectorizer 객체의 위치
        반환값:
            ReviewVectorizer의 인스턴스
    """
    with open(vectorizer_filepath) as fp : 
      return ReviewVectorizer.from_serializable(json.load(fp))

  def save_vectorizer(self, vectorizer_filepath) : 
    """ ReviewVectorizer 객체를 json 형태로 디스크에 저장합니다
        
        매개변수:
            vectorizer_filepath (str): ReviewVectorizer 객체의 저장 위치
    """
    with open(vectorizer_filepath, 'w') as fp :
      json.dump(self._veotorizer._to_serializable(), fp)

  def get_veotorizer(self) : 
    """ 벡터 변환 객체를 반환합니다 """
    return self._vectorizer

  def set_split(self, split='train') : 
    """ 데이터프레임에 있는 열을 사용해 분할 세트를 선택합니다 
        
        매개변수:
            split (str): "train", "val", "test" 중 하나
    """
    self._target_split = split
    self._target_df, self._target_size = self._lookup_dict[split]

  def __len__(self) : 
    return self._target_size

  def __getitem__(self, index) : 
    """ 파이토치 데이터셋의 주요 진입 메서드
        
        매개변수:
            index (int): 데이터 포인트의 인덱스
        반환값:
            데이터 포인트의 특성(x_data)과 레이블(y_target)로 이루어진 딕셔너리
    """
    row = self._target_df.iloc[index]
    review_vector = self._vectorizer.vectorize(row.review)
    rating_index = self._vectorizer.rating_vocab.lookuptoken(row.rating)

    return {'X_data' : review_vector,
            'y_target' : rating_index}

  def get_num_batches(self, batch_size) : 
    """ 배치 크기가 주어지면 데이터셋으로 만들 수 있는 배치 개수를 반환합니다
        
        매개변수:
            batch_size (int)
        반환값:
            배치 개수
    """
    return len(self) // batch_size



### Vocabulary : 토큰을 정수로 매핑하기
- UNK(unknown) : 훈련에서 본 적이 없는 토큰 처리 가능

In [ ]:
class Vocabulary(object) : 
  """ 매핑을 위해 텍스트를 처리하고 어휘 사전을 만드는 클래스 """
  def __init__(self, token_to_idx=None, add_unk=True, unk_token='<UNK>') : 
    """
        매개변수:
            token_to_idx (dict): 기존 토큰-인덱스 매핑 딕셔너리
            add_unk (bool): UNK 토큰을 추가할지 지정하는 플래그
            unk_token (str): Vocabulary에 추가할 UNK 토큰
    """
    if token_to_idx is None : 
      token_to_idx = {}
    self._token_to_idx = token_to_idx